<a href="https://colab.research.google.com/github/AhmedMuoawad/Data-Transformation-Task/blob/master/Data_Transformation_Task_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<br>

### Read Dataset
Let's import the necessary liberaries.

In [0]:
import numpy as np 
import pandas as pd
#from plydata import define, query, if_else
import nltk 
import sklearn
import spacy 
from tqdm import tqdm 
import os 
import re
import seaborn as sns
import matplotlib.pyplot as plt 
import warnings

%matplotlib inline
warnings.filterwarnings('ignore')

### Loading The Dataset 
In this cell, I am using Pandas Lib to reading the flat file by using __'read_csv' function__.

In [2]:
from google.colab import files
import io

# First, upload the file 
uploaded = files.upload()

# Second, Let's read a dataset
df = pd.read_csv(io.BytesIO(uploaded['task_data.csv']))

# Let's show the head of dataset
print(df.head())

Saving task_data.csv to task_data (1).csv
                                    DESC  PRECISION  VALUE
0                 Units (in 5000 PIECES)          3    270
1      Units (any promo) (in 400 PIECES)          2     10
2       Units (no promo) (in 200 PIECES)          6    840
3      Units (tpr only) (in 1300 PIECES)          2    100
4  Units (on pack only) (in 2300 PIECES)          8     10


### Data Transformation ...
Let's start to do some transformation to dataset 

In [3]:
# Let's create new aggregation column 'RATE' = (value * 10)^precision
df['RATE'] = (df['VALUE']*10)**df['PRECISION']
df.head()

,DESC,PRECISION,VALUE,RATE
0,Units (in 5000 PIECES),3,270,19683000000
1,Units (any promo) (in 400 PIECES),2,10,10000
2,Units (no promo) (in 200 PIECES),6,840,-1762523724700975104
3,Units (tpr only) (in 1300 PIECES),2,100,1000000
4,Units (on pack only) (in 2300 PIECES),8,10,10000000000000000


Let's see from last output in RATE column that is no perfect to represent the numbers,
So I decide to create the function to convert these numbers and inhance it's representation.

In [0]:
# Let's create the function to convert the number with units 
def human_format(num, precision=2, suffixes=['', 'K', 'M', 'G', 'T', 'P']):
    '''
    pram num : input number that want to convert it 
    pram precision : number of digits after dot mark in float number 
    pram suffixes : marks that determine the unit of number
    '''
    m = sum([abs(num/1000.0**x) >= 1 for x in range(1, len(suffixes))])
    return f'{num/1000.0**m:.{precision}f}{suffixes[m]}'

#print('the answer is %s' % human_format(7454538))  # prints 'the answer is 7.45M'

In [0]:
# Apply human_fprmat function on RATE column in dataset 
df.RATE = df.RATE.apply(human_format)

In [6]:
# Let's see our Dataset After this update
df.head(10)

,DESC,PRECISION,VALUE,RATE
0,Units (in 5000 PIECES),3,270,19.68G
1,Units (any promo) (in 400 PIECES),2,10,10.00K
2,Units (no promo) (in 200 PIECES),6,840,-1762.52P
3,Units (tpr only) (in 1300 PIECES),2,100,1.00M
4,Units (on pack only) (in 2300 PIECES),8,10,10.00P
5,Volume (in 2000 KGS),9,1000,-5527.15P
6,Volume (any promo) (in 1070 KGS),2,100,1.00M
7,Volume (no promo) (in 1040 KGS),6,4301,4541.03P
8,Volume (tpr only) (in 1050 KGS),11,3401,-4956.62P


#### I need to extract the quantity value with units from DESC column 
So , I will create function to extract value and string from text .

In [7]:
# import parser to extract quantity with unit 
from quantulum3 import parser

def quan(st):
    '''
    pram st : input text I want to extract quantity from it .
    '''
    quant= parser.parse(st)
    return quant[0].surface

# create a new column 'QUANTITY_UNIT' to store the quantity with unit 
df['QUANTITY_UNIT'] = df['DESC'].apply(quan)

# Let's see the dataset after updates 
df.head(10)

,DESC,PRECISION,VALUE,RATE,QUANTITY_UNIT
0,Units (in 5000 PIECES),3,270,19.68G,5000 PIECES
1,Units (any promo) (in 400 PIECES),2,10,10.00K,400 PIECES
2,Units (no promo) (in 200 PIECES),6,840,-1762.52P,200 PIECES
3,Units (tpr only) (in 1300 PIECES),2,100,1.00M,1300 PIECES
4,Units (on pack only) (in 2300 PIECES),8,10,10.00P,2300 PIECES
5,Volume (in 2000 KGS),9,1000,-5527.15P,2000 KGS
6,Volume (any promo) (in 1070 KGS),2,100,1.00M,1070 KGS
7,Volume (no promo) (in 1040 KGS),6,4301,4541.03P,1040 KGS
8,Volume (tpr only) (in 1050 KGS),11,3401,-4956.62P,1050 KGS


#### I Create the two columns [QUANTITY, UNIT] , Then use either seprate column to do some transformation . 

In [8]:
# Create the new Two columns 
df['Quantity'], df['UNIT'] = df['QUANTITY_UNIT'].str.split(' ', 1).str

# Let's see the dataset after updates 
df.head(10)

,DESC,PRECISION,VALUE,RATE,QUANTITY_UNIT,Quantity,UNIT
0,Units (in 5000 PIECES),3,270,19.68G,5000 PIECES,5000,PIECES
1,Units (any promo) (in 400 PIECES),2,10,10.00K,400 PIECES,400,PIECES
2,Units (no promo) (in 200 PIECES),6,840,-1762.52P,200 PIECES,200,PIECES
3,Units (tpr only) (in 1300 PIECES),2,100,1.00M,1300 PIECES,1300,PIECES
4,Units (on pack only) (in 2300 PIECES),8,10,10.00P,2300 PIECES,2300,PIECES
5,Volume (in 2000 KGS),9,1000,-5527.15P,2000 KGS,2000,KGS
6,Volume (any promo) (in 1070 KGS),2,100,1.00M,1070 KGS,1070,KGS
7,Volume (no promo) (in 1040 KGS),6,4301,4541.03P,1040 KGS,1040,KGS
8,Volume (tpr only) (in 1050 KGS),11,3401,-4956.62P,1050 KGS,1050,KGS


#### I want devide Value Column by Quantity Column 
#### Let's see ....

In [9]:
# Let's Covert the data type for Quantity column that allow me to do some math op
df['Quantity'] = df.Quantity.astype(int)

# Cheak the data type for quan column
df.dtypes

# create a new column to store the resuls of devide value / quantity 
df['VALUE_BY_QUANTITY'] = np.round(df.VALUE / df.Quantity,3)

# Let's see the dataset after updates 
df.head(10)

# Create the transformation file before unpivot units using value column 
#df.to_csv('Trans_data_before_UNPIVOT_UNITS.csv')

,DESC,PRECISION,VALUE,RATE,QUANTITY_UNIT,Quantity,UNIT,VALUE_BY_QUANTITY
0,Units (in 5000 PIECES),3,270,19.68G,5000 PIECES,5000,PIECES,0.054
1,Units (any promo) (in 400 PIECES),2,10,10.00K,400 PIECES,400,PIECES,0.025
2,Units (no promo) (in 200 PIECES),6,840,-1762.52P,200 PIECES,200,PIECES,4.200
3,Units (tpr only) (in 1300 PIECES),2,100,1.00M,1300 PIECES,1300,PIECES,0.077
4,Units (on pack only) (in 2300 PIECES),8,10,10.00P,2300 PIECES,2300,PIECES,0.004
5,Volume (in 2000 KGS),9,1000,-5527.15P,2000 KGS,2000,KGS,0.500
6,Volume (any promo) (in 1070 KGS),2,100,1.00M,1070 KGS,1070,KGS,0.093
7,Volume (no promo) (in 1040 KGS),6,4301,4541.03P,1040 KGS,1040,KGS,4.136
8,Volume (tpr only) (in 1050 KGS),11,3401,-4956.62P,1050 KGS,1050,KGS,3.239


#### I want to unpivot units column using value column values

In [10]:
# creating a new DataFrame to store Transformation updates by unpivot unit column
df1 = df.pivot(index ='DESC', columns ='UNIT',
               values =['VALUE', 'PRECISION', 'RATE',
                        'QUANTITY_UNIT', 'Quantity', 'VALUE_BY_QUANTITY']) 

# Let's see the dataset after updates 
df1.head(10)

# Create the final transformation file after unpivot units using value column 
#df.to_csv('Trans_data_after_UNPIVOT_UNITS.csv')

VALUE         ... VALUE_BY_QUANTITY       
UNIT                                    KGS PIECES  ...               KGS PIECES
DESC                                                ...                         
Units (any promo) (in 400 PIECES)       NaN     10  ...               NaN  0.025
Units (in 5000 PIECES)                  NaN    270  ...               NaN  0.054
Units (no promo) (in 200 PIECES)        NaN    840  ...               NaN    4.2
Units (on pack only) (in 2300 PIECES)   NaN     10  ...               NaN  0.004
Units (tpr only) (in 1300 PIECES)       NaN    100  ...               NaN  0.077
Volume (any promo) (in 1070 KGS)        100    NaN  ...             0.093    NaN
Volume (in 2000 KGS)                   1000    NaN  ...               0.5    NaN
Volume (no promo) (in 1040 KGS)        4301    NaN  ...             4.136    NaN
Volume (tpr only) (in 1050 KGS)        3401    NaN  ...             3.239    NaN

[9 rows x 12 columns]